In [2]:
import pandas as pd, numpy as np, dataprep as dp
import torch
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("/Users/florian/Documents/github/study/IoT/IoT/aggregated_hourly.csv")
a0 = ["hka-aqm-a017", "hka-aqm-a014"]
a1 = ["hka-aqm-a101", "hka-aqm-a102", "hka-aqm-a103", "hka-aqm-a106", "hka-aqm-a107", "hka-aqm-a108", "hka-aqm-a111", "hka-aqm-a112"]

In [3]:
df_cutoff = df.copy
df_cutoff = dp.cutoff_data(df, "2023-01-13", "2023-03-28")
df_mean = dp.build_lvl_df(df_cutoff, a1, ["tmp", "hum", "CO2", "VOC"], reset_ind= True)
df_mean.head()

,tmp,hum,CO2
0,23.0950,34.000,573.0
1,23.1125,33.840,545.0
2,23.0950,33.865,532.0
3,23.1650,33.825,516.0
4,23.2725,33.780,507.0


In [4]:
X = df_mean.to_numpy()
y = df_mean["tmp"].shift(-1).to_numpy()
X.shape, y.shape

((1775, 3), (1775,))

In [5]:
X_train, X_test, y_train, y_test = dp.train_test_split(X, y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([1748, 3]),
 torch.Size([1748]),
 torch.Size([27, 3]),
 torch.Size([27]))

In [6]:
X_train_new, X_test_new = dp.format_tensor(X_train), dp.format_tensor(X_test)
X_train = X_train_new
X_test = X_test_new
y_train = y_train[:-1]#.unsqueeze(1)
y_test = y_test[:-1]#.unsqueeze(1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([1747, 48, 3]),
 torch.Size([26, 48, 3]),
 torch.Size([1747]),
 torch.Size([26]))

In [7]:
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [20]:
from foo import RNN

RNN_model = RNN(input_size=X_train.shape[2], output_size=1, hidden_size=100, num_layers=1)

In [21]:
from torch.optim import lr_scheduler 

num_epochs = 50
learning_rate = 0.001
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(RNN_model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma= 0.99)

RNN_model.train()
#labels = []
#outputs = []
for epoch in range(num_epochs):
    for i, (features, target) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)
        output = RNN_model(features)
        target = target.unsqueeze(1)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        #labels.extend(target.detach().numpy()[0])
        #outputs.extend(output.detach().numpy()[0])
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
    scheduler.step()
print(scheduler.get_last_lr())

Epoch [1/50], Step [10/28], Loss: 427.4080
Epoch [1/50], Step [20/28], Loss: 380.2815
Epoch [2/50], Step [10/28], Loss: 330.8149
Epoch [2/50], Step [20/28], Loss: 290.5136
Epoch [3/50], Step [10/28], Loss: 222.9423
Epoch [3/50], Step [20/28], Loss: 205.5428
Epoch [4/50], Step [10/28], Loss: 162.2303
Epoch [4/50], Step [20/28], Loss: 150.9689
Epoch [5/50], Step [10/28], Loss: 114.5232
Epoch [5/50], Step [20/28], Loss: 108.3828
Epoch [6/50], Step [10/28], Loss: 80.4980
Epoch [6/50], Step [20/28], Loss: 71.4572
Epoch [7/50], Step [10/28], Loss: 56.8741
Epoch [7/50], Step [20/28], Loss: 45.2910
Epoch [8/50], Step [10/28], Loss: 37.0541
Epoch [8/50], Step [20/28], Loss: 29.9875
Epoch [9/50], Step [10/28], Loss: 24.3551
Epoch [9/50], Step [20/28], Loss: 21.2684
Epoch [10/50], Step [10/28], Loss: 17.5211
Epoch [10/50], Step [20/28], Loss: 13.5375
Epoch [11/50], Step [10/28], Loss: 13.0279
Epoch [11/50], Step [20/28], Loss: 9.5234
Epoch [12/50], Step [10/28], Loss: 7.2736
Epoch [12/50], Step [

In [22]:
RNN_model.eval()  # Set the model to evaluation mode

test_features, test_targets = next(iter(test_loader))  # Get a batch of test data
test_targets = test_targets.unsqueeze(1)  # Expand target to match the output shape

with torch.no_grad():  # Disable gradient computation
    predictions = RNN_model(test_features)  # Make predictions

# Calculate the mean squared error of the predictions
test_loss = criterion(predictions, test_targets)
print('Test Loss: {:.4f}'.format(test_loss.item()))

Test Loss: 1.0171


In [23]:
predictions

tensor([[20.9481],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9485],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486],
        [20.9486]])

In [19]:
import plotly.graph_objects as go

def plt_pred(test_targets, predictions):
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=test_targets, mode='lines', name='test_targets'))
    fig.add_trace(go.Scatter(y=predictions, mode='lines', name='Predictions'))
    fig.update_layout(title='test_targets vs predictions', xaxis_title='Index', yaxis_title='Value')
    fig.show()

plt_pred(test_targets.view(-1).tolist(), predictions.view(-1).tolist())

In [13]:
#torch.save(LSTM_model.state_dict(), "LSTM_model.pth")